In [ ]:
using PandaRobot
using GuSTO

In [ ]:
robot = PandaBot()
env = Factory()

push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,-0.25,0.30), Vec3f0(.3,0.5,0.01)))
push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,-0.25,0.62), Vec3f0(.3,0.5,0.01)))

push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,-0.26,0.30), Vec3f0(.3,0.01,0.33)))
push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,0.25,0.30), Vec3f0(.3,0.01,0.33)))

th_init = [0;  -0.6; 0.; 1.20*pi; 0.25; 1.10*pi; 0.25*pi+0.001]
th_goal = [0;  +0.1; 0.; 1.20*pi; 0.25; 1.42*pi; 0.25*pi-0.001]

In [ ]:
robot = PandaBot()
env = Factory()
push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,0.1,0.), Vec3f0(.2,0.2,0.3)))
push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,-0.2,0.), Vec3f0(.2,0.2,0.3)))

th_init = [pi/4;  0.50; 0; -0.5*pi; 0.25;  0.5*pi; 0.25*pi-0.001]
th_goal = [-pi/5; 0.65; 0; -0.5*pi; 0.25;  0.5*pi; 0.25*pi+0.001]

In [ ]:
pan = Panda()

state = RigidBodyDynamics.MechanismState(pan.mechanism)
world_frame = RigidBodyDynamics.root_frame(pan.mechanism)

set_configuration!(state,th_goal)

EE_id = 7 + 2
EE_link = RigidBodyDynamics.bodies(pan.mechanism)[EE_id]
EE_link_frame = RigidBodyDynamics.default_frame(EE_link)
EE_link_point = RigidBodyDynamics.Point3D(EE_link_frame, 0.0,0.0,0.)
p_EE = RigidBodyDynamics.Spatial.transform(state,EE_link_point,world_frame)
p_EE.v

world_frame = RigidBodyDynamics.root_frame(pan.mechanism)
EE_path = RigidBodyDynamics.path(pan.mechanism, root_body(pan.mechanism), EE_link)
J_pEE_joint = point_jacobian(state,EE_path, RigidBodyDynamics.Spatial.transform(state,EE_link_point,world_frame))
J_pEE_joint.J[:,1:robot.num_joints];

In [ ]:
model = PandaKin()

N = 81
tf_guess = 100.

x_init, x_goal = zeros(model.x_dim), zeros(model.x_dim)
for i in 1:robot.num_joints
    x_init[2*i-1:2*i] = [cos(th_init[i]); sin(th_init[i])]
    x_goal[2*i-1:2*i] = [cos(th_goal[i]); sin(th_goal[i])]
end

r_EE_inWorldFrame = get_EE_position(x_goal,robot) + [0.01;-0.02;0.02]
model.p_EE_goal = r_EE_inWorldFrame

In [ ]:
PD = ProblemDefinition(robot, model, env, x_init, x_goal)
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_so1, "Gurobi", OutputFlag=0)

In [ ]:
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.successful
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec
@show TOSgusto.SCPS.J_true[end];

In [ ]:
joint_traj = zeros(robot.num_joints,N)
for k in 1:N
    joint_traj[:,k] = get_configuration(TOSgusto.traj.X[:,k],model)
end
verify_joint_limits(TOSgusto.SCPS.traj, TOSgusto.SCPS.SCPP)

In [ ]:
r_EE_f = get_EE_position(TOSgusto.traj.X[:,end],robot)
println("Distance between r_EE_goal and solution is ", norm(r_EE_f-r_EE_inWorldFrame))
abs.(r_EE_f-r_EE_inWorldFrame)
[r_EE_f r_EE_inWorldFrame]

In [ ]:
for idx in 1:model.num_joints
    PyPlot.figure()
    PyPlot.plot(joint_traj[idx,:], linestyle="--", color="k")
    PyPlot.plot(robot.q_max[idx]*ones(N), linestyle="-.", color="r")
    PyPlot.plot(robot.q_min[idx]*ones(N), linestyle="-.", color="r")
end

In [ ]:
norms = zeros(model.num_joints,N)
for k in 1:N
   for idx in 1:model.num_joints
        norms[idx,k] = sqrt(TOSgusto.traj.X[2*idx-1,k]^2 + TOSgusto.traj.X[2*idx,k]^2)
    end
end

for idx in 1:model.num_joints
    PyPlot.plot(norms[idx,:])
end

PyPlot.figure()
for k in 1:model.num_joints
    PyPlot.plot(joint_traj[k,:])
end

In [ ]:
using MeshCat
using MeshCatMechanisms

using RigidBodySim
using RigidBodyDynamics

vis = Visualizer()

pd = robot.pan
mvis = MechanismVisualizer(
    pd.mechanism,
    URDFVisuals(PandaRobot.urdfpath(), package_path=[dirname(dirname(PandaRobot.urdfpath()))]),
    vis);
# 
vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(0.,0.,0.,0.5))))
end

EE_box = HyperRectangle(Vec3f0(model.p_EE_goal-model.p_EE_goal_delta_error), Vec3f0(2*model.p_EE_goal_delta_error*ones(3)))
setobject!(vis[:obstacles][Symbol(string("obs",length(env.obstacle_set)+1))], 
        Object(EE_box,MeshBasicMaterial(color=RGBA(1.,1.,1.,0.5))))

q = Vector{Array{Float64,1}}(0)
for k in 1:N
    push!(q, joint_traj[:,k]) 
end

setanimation!(mvis, 1:length(q), q)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)
#sleep(1)